In [3]:
import pandas as pd
import sqlite3
import pyodbc

In [5]:
Northwind = {
    'servername': r'MAX-PC-DEC-2023\SQLEXPRESS',
    'database': 'northwind'
}

Adventure = {
    'servername': r'MAX-PC-DEC-2023\SQLEXPRESS',
    'database': 'AdventureWorks2019'
}

aenc_connection = sqlite3.connect("../data/aenc.sqlite")
adventure_connection = pyodbc.connect('DRIVER={SQL Server};SERVER=' + Adventure['servername'] + ';DATABASE=' + Adventure['database'] + ';Trusted_Connection=yes')
northwind_connection = pyodbc.connect('DRIVER={SQL Server};SERVER=' + Northwind['servername'] + ';DATABASE=' + Northwind['database'] + ';Trusted_Connection=yes')

In [7]:
# Query to fetch data from NorthWind database
northwind_query = """
SELECT 
    EmployeeID AS Id,
    'NW' AS Database,
    FirstName,
    LastName,
    Address,
    City,
    Region,
    PostalCode AS Postalcode,
    Country,
    HomePhone AS Phonenumber,
    NULL AS Sex,  -- Assuming sex is not available
    HireDate AS Startdate,
    ReportsTo AS ManagerId,
    NULL AS TerminationDate  -- Assuming termination date is not available
FROM Employees
"""
northwind_data = pd.read_sql_query(northwind_query, northwind_connection)

C:\Users\Maxhb\AppData\Local\Temp\ipykernel_34052\2192366749.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  northwind_data = pd.read_sql_query(northwind_query, northwind_connection)


DatabaseError: Execution failed on sql '
SELECT 
    EmployeeID AS Id,
    'NW' AS Database,
    FirstName,
    LastName,
    Address,
    City,
    Region,
    PostalCode AS Postalcode,
    Country,
    HomePhone AS Phonenumber,
    NULL AS Sex,  -- Assuming sex is not available
    HireDate AS Startdate,
    ReportsTo AS ManagerId,
    NULL AS TerminationDate  -- Assuming termination date is not available
FROM Employees
': ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near the keyword 'Database'. (156) (SQLExecDirectW)")

In [ ]:
# Query to fetch data from Aenc database
aenc_query = """
SELECT
    e.EmployeeID AS Id,
    'AENC' AS Database,
    e.FirstName,
    e.LastName,
    e.Address,
    e.City,
    s.StateName AS Region,
    e.PostalCode AS Postalcode,
    e.Country,
    e.PhoneNumber AS Phonenumber,
    e.Sex,
    e.HireDate AS Startdate,
    e.ManagerID AS ManagerId,
    e.TerminationDate
FROM employee e
JOIN state s ON e.StateID = s.StateID
"""
aenc_data = pd.read_sql_query(aenc_query, aenc_connection)

In [ ]:
# Query to fetch data from AdventureWorks database
adventure_query = """
SELECT
    p.BusinessEntityID AS Id,
    'AW' AS Database,
    p.FirstName,
    p.LastName,
    a.AddressLine1 AS Address,
    a.City,
    sp.Name AS Region,
    a.PostalCode AS Postalcode,
    a.CountryRegionCode AS Country,
    pp.PhoneNumber,
    p.Gender AS Sex,
    e.HireDate AS Startdate,
    e.ManagerID AS ManagerId,
    NULL AS TerminationDate  -- Assuming termination date is not directly available
FROM Person.Person p
JOIN HumanResources.Employee e ON p.BusinessEntityID = e.BusinessEntityID
JOIN Person.Address a ON p.BusinessEntityID = a.BusinessEntityID
JOIN Person.StateProvince sp ON a.StateProvinceID = sp.StateProvinceID
JOIN Person.PersonPhone pp ON p.BusinessEntityID = pp.BusinessEntityID
"""
adventure_data = pd.read_sql_query(adventure_query, adventure_connection)

In [1]:
# Combine all data into a single DataFrame
combined_data = pd.concat([northwind_data, aenc_data, adventure_data], ignore_index=True)

# Display the combined data
print(combined_data.head())

# Optionally, you can save this DataFrame to a new SQLite database or a CSV file
# combined_data.to_csv('combined_employees.csv', index=False)


DatabaseError: Execution failed on sql '
SELECT
    e.EmployeeID AS Id,
    'AENC' AS Database,
    e.FirstName,
    e.LastName,
    e.Address,
    e.City,
    s.StateName AS Region,
    e.PostalCode AS Postalcode,
    e.Country,
    e.PhoneNumber AS Phonenumber,
    e.Sex,
    e.HireDate AS Startdate,
    e.ManagerID AS ManagerId,
    e.TerminationDate
FROM employee e
JOIN state s ON e.StateID = s.StateID
': no such column: e.EmployeeID